Install Spark and pyspark on Mac OSX: https://medium.com/@yajieli/installing-spark-pyspark-on-mac-and-fix-of-some-common-errors-355a9050f735

Get some BIG test data: https://cs.fit.edu/~mmahoney/compression/textdata.html (Wikipedia dumps)

In [1]:
!curl https://cs.fit.edu/~mmahoney/compression/enwik8.bz2 -o enwik8.bz2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  242M  100  242M    0     0   457k      0  0:09:02  0:09:02 --:--:--  157k 1355k      0  0:03:02  0:00:33  0:02:29 1827k  0     0   870k      0  0:04:44  0:01:19  0:03:25  255k    0     0   781k      0  0:05:17  0:01:30  0:03:47  182k0k      0  0:05:39  0:01:38  0:04:01  110k 0:02:11  0:04:44  248kM    0     0   595k      0  0:06:56  0:02:12  0:04:44  296k 80.8M    0     0   581k      0  0:07:06  0:02:22  0:04:44  433k0     0   530k      0  0:07:47  0:02:44  0:05:03  206k   484k      0  0:08:32  0:03:17  0:05:15  142k28k      0  0:09:38  0:04:29  0:05:09  102k 422k      0  0:09:47  0:04:37  0:05:10  202k17k      0  0:09:53  0:04:42  0:05:11  166k     0   413k      0  0:09:59  0:04:48  0:05:11  224k55  133M    0     0   385k      0  0:10:43  0:05:55  0:04:48  213k 146M    0     0   375k      0  0:11:00  0:06:38  0:04:22  357k 2

In [2]:
!bunzip2 -v enwik8.bz2

  enwik9.bz2: done


In [3]:
from pyspark import SparkContext
sc = SparkContext()

In [4]:
def pprint_plan(rdd):
    for x in rdd.toDebugString().decode().split('\n'):
        print(x)

In [7]:
%%time
textfile = sc.textFile('enwik9')

CPU times: user 1.18 ms, sys: 1.4 ms, total: 2.57 ms
Wall time: 30.2 ms


In [8]:
pprint_plan(textfile)

(30) enwik9 MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0 []
 |   enwik9 HadoopRDD[2] at textFile at NativeMethodAccessorImpl.java:0 []


In [9]:
%%time
words =     textfile.flatMap(lambda line: line.split(" "))
not_empty = words.filter(lambda x: x!='') 
key_values= not_empty.map(lambda word: (word, 1)) 
counts=     key_values.reduceByKey(lambda a, b: a + b)

CPU times: user 11.6 ms, sys: 3.93 ms, total: 15.5 ms
Wall time: 49.6 ms


In [10]:
pprint_plan(words)

(30) PythonRDD[8] at RDD at PythonRDD.scala:53 []
 |   enwik9 MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0 []
 |   enwik9 HadoopRDD[2] at textFile at NativeMethodAccessorImpl.java:0 []


In [11]:
pprint_plan(not_empty)

(30) PythonRDD[9] at RDD at PythonRDD.scala:53 []
 |   enwik9 MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0 []
 |   enwik9 HadoopRDD[2] at textFile at NativeMethodAccessorImpl.java:0 []


In [12]:
pprint_plan(key_values)

(30) PythonRDD[10] at RDD at PythonRDD.scala:53 []
 |   enwik9 MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0 []
 |   enwik9 HadoopRDD[2] at textFile at NativeMethodAccessorImpl.java:0 []


In [13]:
pprint_plan(counts)

(30) PythonRDD[11] at RDD at PythonRDD.scala:53 []
 |   MapPartitionsRDD[7] at mapPartitions at PythonRDD.scala:133 []
 |   ShuffledRDD[6] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(30) PairwiseRDD[5] at reduceByKey at <timed exec>:4 []
    |   PythonRDD[4] at reduceByKey at <timed exec>:4 []
    |   enwik9 MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0 []
    |   enwik9 HadoopRDD[2] at textFile at NativeMethodAccessorImpl.java:0 []


In [14]:
%%time
counts.collect()

CPU times: user 2.2 s, sys: 460 ms, total: 2.66 s
Wall time: 1min 26s


[('<revision>', 243426),
 ('Samoa]]{{R', 1),
 ('<id>15898943</id>', 1),
 ('<id>15898945</id>', 1),
 ('of', 4805157),
 ('unnecessary', 719),
 ('rather', 22574),
 ('free', 14232),
 ('especially', 20077),
 ('before', 49458),
 ('Private', 1527),
 ('Rothbard|Rothbard]]', 1),
 ('[[Zeno', 109),
 ('religious', 13228),
 ("d'Arce", 1),
 ('no', 110749),
 ('Revolution]],', 785),
 ('have', 359712),
 ('&quot;[[Property', 2),
 ('vision', 2147),
 ('[[mutualism]]', 8),
 ('movements,', 628),
 ('discussion', 4650),
 ('institutions', 3181),
 ("egoists'", 1),
 ('conflict.', 806),
 ('property.', 861),
 ('interested', 3293),
 ('right', 17472),
 ('Millwood,', 11),
 ('1848.', 114),
 ('links', 55638),
 ('signficiant.', 1),
 ('two', 147750),
 ('{{main|Anarchist', 1),
 ('away', 11750),
 ('[http://nefac.net/node/157]', 1),
 ('illustrated', 1645),
 ('below)', 910),
 ('manifesto', 251),
 ('==Propaganda', 6),
 ('outspoken', 549),
 ('{{main|Propaganda', 1),
 ('[[consensus]]', 49),
 ('desirable', 849),
 ('[[general', 7

In [15]:
%%time
Count=counts.count()  # Count = the number of different words
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y) # 
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=8868878, total words=129341541, mean no. occurances per word=14.58
CPU times: user 17.6 ms, sys: 5.23 ms, total: 22.9 ms
Wall time: 10 s
